### This notebook well analyze the cleaned datasets
#### The input data comes from the cleaned CSV files produced in the data_cleaning notebook.

In [1]:
# import pandas
import pandas as pd

In [2]:
# Loading the cleaned csv datasets

items = pd.read_csv("../output/items_cleaned.csv")
orders = pd.read_csv("../output/orders_cleaned.csv")
customers = pd.read_csv("../output/customers_cleaned.csv")

In [3]:
# Standardize column data types in the cleaned items dataset to ensure accurate analysis


items["shipping_limit_date"] = pd.to_datetime(items["shipping_limit_date"])
items = items.astype({
    "order_id" : str,
    "order_item_id" : int,
    "product_id" : str,
    "seller_id" : str,
    "price" : float,
    "freight_value" : float
})

In [4]:
# Standardize column data types in the cleaned orders dataset to ensure accurate analysis


orders = orders.astype({
    "order_id" : str,
    "customer_id" : str,
    "order_status" : str
})
# create a function to speed up the process
def date_time(column) : 
    orders[column] = pd.to_datetime(orders[column])

date_time("order_purchase_timestamp")
date_time("order_approved_at")
date_time("order_delivered_carrier_date")
date_time("order_delivered_customer_date")
date_time("order_estimated_delivery_date")

In [5]:
# Standardize column data types in the cleaned customers dataset to ensure accurate analysis


customers = customers.astype({
    "customer_id" : str,
    "customer_unique_id" : str,
    "customer_zip_code_prefix" : str,
    "customer_city" : str,
    "customer_state" : str
})
#   here we set "customer_zip_code_prefix" as a string,
# because zip codes are stored as strings because they are identifiers,
# and some values in it can start with a 0

### The total number of orders 

In [6]:
order_count = len(orders)
order_count

99281

> There are 99,281 orders

### Orders by status 

In [7]:
order_status_count = orders["order_status"].value_counts()
order_status_count 

order_status
delivered      96464
shipped         1107
unavailable      609
canceled         484
invoiced         314
processing       301
approved           2
Name: count, dtype: int64

> 96464 orders are delivered, while a smaller portion of orders varies in status

### Undelivered orders

In [8]:
# Counting with .loc[] how many values are in orders["order_status"] that are not delivered or canceled 
undelivered_orders = orders.loc[~orders["order_status"].isin(["canceled","delivered"]), "order_status"].value_counts().sum()
undelivered_orders

np.int64(2333)

> There's 2333 undelivered orders 

### Average delivery time 

In [9]:
delivered = orders[orders["order_status"] == "delivered"] 
avg_delivery_time = delivered["order_delivered_customer_date"] - delivered["order_purchase_timestamp"]
avg_delivery_time = avg_delivery_time.mean()
avg_delivery_time

Timedelta('12 days 13:23:40.390437090')

> The average delivery time is around 12 and a half days

### Calculating the total number of orders per city/state 

#### Per city

In [10]:
# merging orders and customers
orders_customers = orders.merge(
    customers,
    on = "customer_id",
    how = "left"
)
order_count_city = orders_customers["customer_city"].value_counts().reset_index(name="orders").sort_values(by="orders",ascending=False)
order_count_city.head(10)

,customer_city,orders
0,sao paulo,15511
1,rio de janeiro,6870
2,belo horizonte,2768
3,brasilia,2128
4,curitiba,1519
5,campinas,1440
6,porto alegre,1377
7,salvador,1245
8,guarulhos,1187
9,sao bernardo do campo,937


> - The top proferming cities are : sao paulo,rio de janeiro,belo horizonte,brasilia,curitiba
> - The top proferming city is sao paulo with 15511 orders 

#### Per state

In [35]:
order_count_state = orders_customers["customer_state"].value_counts().reset_index(name="orders").sort_values(by="orders",ascending=False)
order_count_state.head(10)

,customer_state,orders
0,SP,41667
1,RJ,12832
2,MG,11619
3,RS,5456
4,PR,5038
5,SC,3631
6,BA,3378
7,DF,2137
8,ES,2031
9,GO,2018


>  - Top proferming states are : SP,RJ,MG,RS,PR
>  - The state Sao Paulo(SP) has the highest amount of orders

### Calculating the total revenue 

#### Calculating the total revenue without the freight value

In [12]:
orders_items = orders.merge(
    items,
    on = "order_id",
    how = "left" 
)
total_revenue = orders_items["price"].sum()
print(f"{total_revenue} $")

13589973.270000001 $


> The total revenue without the freight value is 13589973.27$

#### calculating the total revenue with the freight value

In [13]:
total_revenue_with_freight = (orders_items["price"] + orders_items["freight_value"]).sum()
print(f"{total_revenue_with_freight} $")

15841598.639999997 $


> The total revenue with the freight value is 15841598.63 $

### Calculating the total number of delivery delays 

In [14]:
delivery_delays = orders[orders["order_estimated_delivery_date"] < orders["order_delivered_customer_date"]]
len(delivery_delays)

7827

> Of all the deliveries, 7,827 were found to be delayed

### calculating trends over time (month period)

In [15]:
orders_by_time = orders.set_index("order_purchase_timestamp")
monthly_orders_trend = orders_by_time.resample("ME")["order_id"].count()
monthly_orders_trend.sort_values(ascending=False).head(10)

order_purchase_timestamp
2017-11-30    7535
2018-01-31    7268
2018-03-31    7208
2018-04-30    6939
2018-05-31    6872
2018-02-28    6724
2018-08-31    6459
2018-07-31    6291
2018-06-30    6167
2017-12-31    5666
Name: order_id, dtype: int64

> The top trending months are :
> 1. november 2017 with the most orders.
> 2. january 2018
> 3. march 2018

### Top months by orders

In [16]:
orders["month"] = orders["order_purchase_timestamp"].dt.month
orders_per_month = orders.groupby("month").size().reset_index(name = "orders_count").sort_values(by="orders_count",ascending=False)
orders_per_month

,month,orders_count
7,8,10784
4,5,10563
6,7,10312
2,3,9888
5,6,9408
3,4,9339
1,2,8490
0,1,8065
10,11,7535
11,12,5667


> - The month with the highest amount of orders is August
> - The month with the lowest amount of orders is September

### Top years by orders

In [17]:
orders["year"] = orders["order_purchase_timestamp"].dt.year
orders_per_year = orders.groupby("year").size().reset_index(name = "orders_count").sort_values(by="orders_count",ascending=False)
orders_per_year

,year,orders_count
2,2018,53929
1,2017,45029
0,2016,323


> - The year with the most amount of orders is 2018
> - We can see that the years descend like this 2018>2017>2016 showing a growing pattern

### Top products by revenue

In [18]:
top_products_by_revenue = orders_items.groupby("product_id")["price"].sum().reset_index(name = "product_revenue").sort_values(by = "product_revenue",ascending = False)
top_products_by_revenue.head(5)

,product_id,product_revenue
24084,bb50f2e236e5eea0100680137654686c,63885.00
14066,6cdd53843498f92890544667809f1595,54730.20
27611,d6160fb7873f184099d9bc95e30376af,48899.34
27037,d1c427060a0f73f6b889a5c7c61f2ac4,47214.51
19740,99a4788cb24856965c36a24e339b6058,43025.56


> The product with the highest revenue is with the ID : *bb50f2e236e5eea0100680137654686c*

### Top sellers by revenue

In [19]:
top_sellers = orders_items.groupby("seller_id")["price"].sum().reset_index(name = "seller_revenue").sort_values(by = "seller_revenue",ascending = False)
top_sellers.head(5)

,seller_id,seller_revenue
857,4869f7a5dfa277a7dca6462dcf3b52b2,229472.63
1013,53243585a1d6dc2643021fd1853d8905,222776.05
881,4a3ca9315b744ce9f8e9374361493884,200472.92
3024,fa1c13f2614d7b5c4749cbc52fecda94,194042.03
1535,7c67e1448b00f6e969d365cea6b010ab,187923.89


> The top seller with the highest revenue is with the ID : *4869f7a5dfa277a7dca6462dcf3b52b2*

### On-time VS Late deliveries 

In [20]:
orders["delivery_status"] = orders["order_delivered_customer_date"] > orders["order_estimated_delivery_date"] 
orders["delivery_status"] = orders["delivery_status"].map({
    True : "Late",
    False : "On-time"
})
On_time_VS_Late_deliveries = orders.groupby("delivery_status").size().reset_index(name="deliveries_count").sort_values(by="deliveries_count",ascending=False)
On_time_VS_Late_deliveries

,delivery_status,deliveries_count
1,On-time,91454
0,Late,7827


> As we can see, the majority of deliveries arrived **on time**, while a much smaller portion of deliveries arrived **late**

### Calculating order count per customer (One-time VS Repeat)

In [21]:
customer_orders = orders_customers.groupby("customer_unique_id").size().reset_index(name = "orders")
customer_orders["status"] = customer_orders["orders"] > 1 
customer_status = customer_orders.value_counts("status").reset_index(name = "customers")
customer_status["status"] = customer_status["status"].map({True : "Repeat", False : "One-time"})
customer_status

,status,customers
0,One-time,93049
1,Repeat,2948


> As we can see, the majority of customers only buy once from the store indicating a low rate of repeat buying

### Top customers by total spend and their contribution (%)

In [22]:
top_customers = orders_items.groupby("customer_id")["price"].sum().reset_index(name="total_spent").sort_values(by = "total_spent",ascending = False)
top_customers["contribution_pct"] = (top_customers["total_spent"] / total_revenue) * 100
top_customers.head(10)

,customer_id,total_spent,contribution_pct
8528,1617b1357756262bfa56ab541c47bc16,13440.0,0.098896
91836,ec5b2ba62e574342386871631fafd3fc,7160.0,0.052686
77396,c6e2731c5b391845f6800c97401a43a9,6735.0,0.049559
94970,f48d464a0baaea338cb25f816991ab1f,6729.0,0.049514
24733,3fd6777bbce08a352fddd04e4a7cc8f6,6499.0,0.047822
2060,05455dfa7cd02f13d132aa7a6a9729c6,5934.6,0.043669
86768,df55c14d1476a9a3467f131269c2477f,4799.0,0.035313
14256,24bbf5fd2f2e1b359ee7de94defc4a15,4690.0,0.034511
87255,e0a2412720e9ea4f26c1ac985f6a7358,4599.9,0.033848
23894,3d979689f636322c62418b6346b1c6d2,4590.0,0.033775


In [23]:
orders_items

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.0,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113259,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,1.0,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,174.90,20.10
113260,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,1.0,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,205.99,65.02
113261,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,1.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59
113262,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,2.0,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59


> As we can see in the output, the top customer with the most orders is with the ID : 8d50f5eadf50201ccdcedfb9e2ac8455

### Customer distribution by location (City/State)

#### By city 

In [24]:
top_cities = orders_customers.groupby("customer_city")["customer_unique_id"].nunique().reset_index(name="customers").sort_values(by = "customers", ascending = False)
top_cities.head(10)

,customer_city,customers
3597,sao paulo,14967
3155,rio de janeiro,6610
453,belo horizonte,2671
558,brasilia,2067
1143,curitiba,1463
707,campinas,1396
2964,porto alegre,1325
3247,salvador,1209
1529,guarulhos,1151
3428,sao bernardo do campo,907


> As we can see in the output, the city that holds the highest amount of customers is **sao paulo**

#### By state

In [25]:
top_states = orders_customers.groupby("customer_state")["customer_unique_id"].nunique().reset_index(name="customers").sort_values(by = "customers", ascending = False)
top_states.head(10)

,customer_state,customers
25,SP,40255
18,RJ,12370
10,MG,11249
22,RS,5271
17,PR,4877
23,SC,3531
4,BA,3276
6,DF,2073
7,ES,1962
8,GO,1952


> As we can see in the output, the state sau-paulo (SP) holds the highest amount of customers

### Top 10 products by orders

In [26]:
top_products = orders_items.groupby("product_id").size().reset_index(name="orders_count").sort_values(by = "orders_count",ascending = False)
top_products.head(10)

,product_id,orders_count
22110,aca2eb7d00ea1a7b8ebd4e68314663af,527
19740,99a4788cb24856965c36a24e339b6058,488
8612,422879e10f46682990de24d770e7f83d,484
7363,389d119b48cf3043d311335e499d9c6b,392
7078,368c6c730842d78016ad823897a372db,388
10838,53759a2ecddad2bb87a079a1f1519f73,373
27037,d1c427060a0f73f6b889a5c7c61f2ac4,343
10865,53b36df67ebb7c41585e8d54d6772e08,323
2794,154e7e31ebfa092203795c972e5804a6,281
8050,3dd2a17168ec895c781a9191c1e95ad7,274


> As we can see in the output, the top product with 527 orders is with the ID : *aca2eb7d00ea1a7b8ebd4e68314663af*

### Sellers with the highest average delay

In [27]:
orders_items["delay"] = (orders_items["order_delivered_customer_date"] - orders_items["order_estimated_delivery_date"]).dt.days
top_sellers_by_delay = orders_items.groupby("seller_id")["delay"].median().reset_index(name="avg_delay").sort_values(by="avg_delay",ascending = False)
top_sellers_by_delay.head(5)

,seller_id,avg_delay
2699,df683dfda87bf71ac3fc63063fba369d,167.0
1745,8e670472e453ba34a379331513d6aab1,35.0
1645,8629a7efec1aab257e58cda559f03ba7,33.0
667,391bbd13b6452244774beff1824006ed,24.0
2292,be1e9e378700cecaa4ebf71433d7915c,23.5


> As we can see in the output, the seller with the highest average delay is with the ID : *df683dfda87bf71ac3fc63063fba369d* with the average delay of 167 days

### Least ordered products

In [32]:
least_ordered_products = orders_items.groupby("product_id").size().reset_index(name="orders_count").sort_values(by="orders_count",ascending=True)
least_ordered_products.head(10)

,product_id,orders_count
10,001c5d71ac6ad696d22315953758fa04,1
8,001b237c0e9bb435f2e54071129237e9,1
32930,ffdde3d63e889c9a9f9ec30d82a4c815,1
18,002c6dab60557c48cfd6c2222ef7fd76,1
32922,ffd246249e3225c13f40b5b91dcaa65a,1
32923,ffd259a48b9b073c942884d0f3659566,1
32927,ffd63ee42a5c8cc5a15a1c8e2aa50011,1
32920,ffcfaba393e8ef71937c6e8421bc2868,1
30,004154251837f6ac124ad4374b3a8148,1
31,0042f1a9a7e0edd1400c6cd0fda065f8,1


> As we can see in the output, the least ordered products were only ordered once

### Calculating how many products were only ordered once

In [29]:
ordered_once = len(least_ordered_products[least_ordered_products["orders_count"] == 1])
ordered_once

18117

> As we can see in the output, there's **18117** products that were only ordered once

### Percentage of revenue from top customers

In [30]:
customer_grouped = orders_items.groupby("customer_id").size().reset_index(name="orders")
top_customers_by_orders = customer_grouped[customer_grouped["orders"] >= customer_grouped["orders"].mean()]
# Calculating the total revenue to compare:
total_revenue = orders_items["price"].sum()

# Calculating the total revenue from top customers to compare : 
revenue_from_top_customers = orders_items[orders_items["customer_id"].isin(top_customers_by_orders["customer_id"])]["price"].sum()

top_customers_revenue_percentage = (revenue_from_top_customers/total_revenue)*100
print(f"{top_customers_revenue_percentage}%")

14.759149117884906%


> Percentage of revenue from the top customers is **14.75%**




### Saving all the variables produced to be used later as a CSV file  

In [36]:
with open("../output/Key_analysis_findings/top_customers_revenue_percentage.txt", "w") as f:
    f.write(f"{top_customers_revenue_percentage}%")
with open("../output/Key_analysis_findings/ordered_once.txt", "w") as f:
    f.write(f"{ordered_once}%")
least_ordered_products.to_csv('../output/Key_analysis_findings/least_ordered_products.csv', index=False)
top_sellers_by_delay.to_csv('../output/Key_analysis_findings/top_sellers_by_delay.csv', index=False)
top_products.to_csv('../output/Key_analysis_findings/top_products.csv', index=False)
top_states.to_csv('../output/Key_analysis_findings/top_states.csv', index=False)
top_cities.to_csv('../output/Key_analysis_findings/top_cities.csv', index=False)
top_customers.to_csv('../output/Key_analysis_findings/top_customers.csv', index=False)
customer_status.to_csv('../output/Key_analysis_findings/customer_status.csv', index=False)
On_time_VS_Late_deliveries.to_csv('../output/Key_analysis_findings/On_time_VS_Late_deliveries.csv', index=False)
top_sellers.to_csv('../output/Key_analysis_findings/top_sellers.csv', index=False)
top_products.to_csv('../output/Key_analysis_findings/top_products.csv', index=False)
top_products_by_revenue.to_csv('../output/Key_analysis_findings/top_products_by_revenue.csv', index=False)
orders_per_year.to_csv('../output/Key_analysis_findings/orders_per_year.csv', index=False)
orders_per_month.to_csv('../output/Key_analysis_findings/orders_per_month.csv', index=False)
monthly_orders_trend.to_csv('../output/Key_analysis_findings/monthly_orders_trend.csv', index=False)
delivery_delays.to_csv('../output/Key_analysis_findings/delivery_delays.csv', index=False)
with open("../output/Key_analysis_findings/total_revenue_with_freight.txt", "w") as f:
    f.write(f"{total_revenue_with_freight}%")
with open("../output/Key_analysis_findings/total_revenue.txt", "w") as f:
    f.write(f"{total_revenue}%")
order_count_state.to_csv('../output/Key_analysis_findings/order_count_state.csv', index=False)
order_count_city.to_csv('../output/Key_analysis_findings/order_count_city.csv', index=False)
with open("../output/Key_analysis_findings/avg_delivery_time.txt", "w") as f:
    f.write(f"{avg_delivery_time}%")
with open("../output/Key_analysis_findings/undelivered_orders.txt", "w") as f:
    f.write(f"{undelivered_orders}%")
order_status_count.to_csv('../output/Key_analysis_findings/order_status_count.csv', index=False)
with open("../output/Key_analysis_findings/order_count.txt", "w") as f:
    f.write(f"{order_count}%")